# Introduction to functional programming in Python
<table style="margin-top:30px; border:none" align="left">
    <tr style="border:none"><td style="border:none"><p>Francesco Bruni</p></td></tr>
    <tr style="border:none" style="width:100%">
        <td style="border:none" style="width:90%"><p>@brunifrancesco</p></td>
    </tr>
</table>


## Who am I

- MSc Student in Telecommunication Engineering @ Poliba
- Despite a Java background, I prefer Python whenever possible
- I'm not a computer scientist

## Agenda

- Why functional programming
    - Everything's object
    - Laziness: why evaluation matters
    - Immutable data
    - Recursion and/or cycles
    - Pattern matching
- Mixing OOP with FP
- FP suitable patterns
- Conclusions


## Disclaimer

<p style="text-align: center; font-size: 40px"><strong>I'm not an hoover seller</strong></p>


# Why functional programming

- Think in terms of **functions**
- Enfatizes using *function evaluation* instead of *state evolution*
- Testing is (more) easy
- A new viewpoint is required

## What is a function?

A **relation** from a set of inputs (X) to a set of possible outputs (Y) where **each** input is related to **exactly** one output.

<p style="text-align:center; font-size:20px;">f: X → Y</p>
<img src="images/function.svg" />

In [1]:
import random
def sum_imperative():
    res = 0
    for n in [random.random() for _ in range(100)]:
        res += n
    return res

def sum_functional():
    return sum(random.random() for _ in range(100))


# Functional features in Python

- Not all functional patterns apply to Python
- Hybrid approach is often requested
- Other libraries needs to be (eventually) integrated



## First class (or high order) functions and 

- Since everything's object
- Functions are objects too (with fields and methods)


In [1]:
def sum_values(*args, **kwargs):
    """
    Sum undefined number of values
    """
    return sum(args)
    
    def __repr__(self):
        return "here"

print("**Doc**: %s" %sum_values.__doc__)
print("**Name**: \n    %s" %sum_values.__name__)




**Doc**: 
    Sum undefined number of values
    
**Name**: 
    sum_values


## High order functions

- Functions accepting functions as params
- Functions returning other functions
- **filter**, **map**, **reduce** (now in *itertools* module)

## Function composition in math

A pointwise application of one function to the result of another to produce a third function. 

Given:
- f : X → Y 
- g : Y → Z 
          
<p style="text-align:center; font-size:20px">$g\circ f$ : X → Z</p>


In [12]:

def mapper_func(lst, reduce_func):
    """
    Apply a function to each member of <lst>
    """
    return map(reduce_func, lst)

for _ in range(0, 100):
    assert list(mapper_func([1,2,3], str))  == ["1", "2", "3"]
    assert list(mapper_func(["1","2","3"], int))  == [1, 2, 3]

In [16]:
def check_if_neg(value):
        """
        Check if values is neg
        """
        return value > 0
    
def filter_negative_numbers(lst):
    """
    Filter negative numbers from <lst>
    """
    return filter(check_if_neg, lst)

assert list(filter_negative_numbers([-3, 4, 5, -10, 20])) == [4, 5, 20]

## Pure functions

- Functions cannot include any assignement statement
- **lambda functions** are pure functions
- No side effects
- What about default param values? 

## λ calculus

- A formal system to fomally analyze functions and their calculus.
- It deals with rewriting functions with simplified terms
- A formal definition of λ function:

<center style="font-size:20px">Λ ::= X |(ΛΛ)|λX.Λ</center>

http://infoteorica.weebly.com/uploads/1/7/8/9/17895653/lambda_rid.pdf

In [17]:
import random
def filter_out(result=[random.random() for _ in range(0, 5)]):
    
    exclude = map(lambda item: item ** 2, range(30))

    result = filter(lambda item: item not in exclude, result)
    
    sorted_result = sorted(result, key=lambda item: str(item)[1])
    
    return map(lambda item: round(item, 2), sorted_result)
    
cast_to_float = lambda number: float(number)
filter_out()

## Practical considerations of λ functions

- Inline functions
- Concise
- No need of defining a *one time* functions
- Overusing them is not a solution
- assigning λ function to a variable is discouraged (PEP8)

## Class for function composition
Handle class creation via a new class syntax, allowing a "static" configuration


In [20]:
from collections.abc import Callable

class ExcludeFunction(Callable):
    def __init__(self, exclude_function):
        self.exclude_function= exclude_function
    
    def __call__(self, value):
        return None if not value else self.exclude_function(value)

# fare esempio con map reduce
result = python_better_approach(size=100)
exclude_function_1 = lambda item: item ** 2
ex_func = ExcludeFunction(exclude_function_1)

result = filter(lambda item: not item == ex_func(item), result)
assert(sorted(result, key=lambda item: str(item)[1]) == filter_out())

NameError: name 'python_better_approach' is not defined

# Immutable vs mutable data structure

- Variables don't **vary** any more
- Will your program still working?

In [91]:
value = 100

def change_f_value(new_f_value=5):
    value = new_f_value
    print("Value in function %s" %value)
    
print("Initialized value %s "%value)
change_f_value()
print("Final value %s "%value)

Initialized value 100 
Value in function 5
Final value 100 


### Function scopes

- Values inside functions exist in the *context* of functions
  - Check this evaluating their own *hash value*
- What if we wanted change the *value* variable?

In [95]:
class Foo:
    def __init__(self, value):
        self.value = value
        
foo_obj = Foo(value=10)

def func(obj):
    obj.value = 3

#print(id(foo_obj))
print(foo_obj.value)
func(foo_obj)
print(foo_obj.value)

4541957512
10
4541957512
4541957512
3


### Data mutation

- *foo_obj* didn't change
- **foo_obj.value** changed!
- So, foo_obj changed or not? If so, can you always determine who changed it?

### Closures

## Immutability

- Let's create a new object, instead of modifying an existing one

In [ ]:
import random
import pprint
from collections import namedtuple

MyObj = namedtuple("MyClassReplacement",("some_string", "my_smart_function"))
o = MyObj(some_string=str(random.random() + 4), my_smart_function=lambda item: float(item)*3)
assert(o.my_smart_function(o.some_string) == float(o.some_string) * 3)

# guess this
#o.some_string = "aaa"



## Strict vs not strict evaluation

- Strict evaluation requires that all operators needs to be evaluated
- Non strict (or lazy) evaluation, evaluates expression if and when requested

# Truth table

In [68]:
import random

generate_random_list = lambda size: [random.choice([True, False]) for _ in range(0, size)]

def all_true_values(lst):
    print("evaluating ALL true values")
    return all(lst)

def any_true_value(lst):
    print("evaluating ANY true values")
    return any(lst)

all_true_values(generate_random_list(size=10)) and any_true_value(generate_random_list(size=10))
print("+++++++++++")
all_true_values(generate_random_list(size=10)) or any_true_value(generate_random_list(size=10))




evaluating ALL true values
+++++++++++
evaluating ALL true values
evaluating ANY true values


True

## Use case: Python iterables structures

- **Iterable**: objects than can be iterated;
- **Iterator**: objects than can be iterated and consumed only once, with two main problems:
    - They track their state in a stateful object
    - Their values are generated *a priori*
    - They aren't lazy
- **Generators**: lazy evaluated data structures:
    - They track state in function, instead of object
    - They don't act as normal functions
    - They generate next member when invoked
    - **asyncio** enjoy this approach :)

In [74]:
def _iter(size=100):
    return iter(python_better_approach(size=size))


def lazy_python_approach(size=100):
     for item in range(10, size):
        if not item % 2 and not str(item).endswith("4"):
            yield item

def lazy_new_python_approach(size=100):
    yield from (r for r in range(10, size) if not r % 2 and not str(r).endswith("4"))

## Recursion vs loop
- Functional programming requires a recursion approach vs loop iteration
- Python suffers by recursion limit
- Python does not offer any tail call optimization

In [7]:
def facti(n):
    if n == 0: return 1
    f= 1
    for i in range(2,n):
        f *= i
    return f

def fact(n):
    if n == 0: return 1
    else: return n*fact(n-1)

## Currying

Multiple arguments functions mapped to single arguments functions

In [19]:
from random import randrange

def mult(a, b, c ):
    def wrapper_1(b):
        def wrapper_2(c):
            return a*b*c
        return wrapper_2(c)
    return wrapper_1(b)


def mult_2(a):
    def wrapper_1(b):
        def wrapper_2(c):
            return a*b*c
        return wrapper_2
    return wrapper_1


assert(mult(2,4,8) == 64)
assert(mult_2(2)(3)(4) == 24)

## Partials
Python provides "partial" functions for manual currying

In [1]:
from functools import reduce
import operator
import random

def two_random_sum():
    return reduce(operator.sum, [random.random()]*2)

def three_random_product():
    return reduce(operator.mul, [random.random()]*3)

def four_random_sub():
    return reduce(operator.sub, [random.random()]*4)

def five_random_pow ():
    return reduce(operator.pow, [random.random()]*5)


In [24]:
def handle_random_numbers(size, function):
    return reduce(function, [random.random()]*size)

two_random_sum = partial(handle_random_numbers, size=2)
three_random_sum = partial(handle_random_numbers, size=3)

two_random_pow = partial(handle_random_numbers, size=2, function=operator.pow)
five_random_product = partial(handle_random_numbers, size=5, function=operator.mul)


## Mixing OOP with FP

- Static methods
- Classes as container
- Decorator pattern
- A revisited strategy pattern

## Decorator Pattern

Return a modified version of a decorated function

In [116]:
from functools import wraps
from functools import partial

def get_ned_data(n):
    def get_doubled_data(func, *args, **kwargs):
        @wraps(func)
        def _inner(*args, **kwargs):
            kwargs["new_param"] = kwargs["some_param"]*n
            return func(*args, **kwargs)
        return _inner
    return get_doubled_data

In [ ]:
@get_ned_data(n=2)
def double_func(*args, **kwargs):
    assert(kwargs["new_param"] == kwargs["some_param"]*2)

@get_ned_data(n=3)
def triple_func(*args, **kwargs):
    assert(kwargs["new_param"] == kwargs["some_param"]*3)
    
double_func(some_param=3)
triple_func(some_param=5)

## FP patterns

- Option pattern
- Memoization
- Actor model

# Monads

- How to handle error in functional programming?
- Practical example: parsing and tranforming data coming from http request
- Python "std" libs have no support
- External resources are required (fn)

In [23]:
from fn.monad import optionable

def get(request, *args, **kwargs):
    @optionable
    def _get_values(data):
        return data.get("values", None)
    _split = lambda item: item.split(",")
    _strip = lambda item: item.replace(" ", "")
    _filter = lambda item: list(filter(lambda i: i, item))
    values = _get_values(request.GET).map(_strip).map(_split).map(_filter).get_or(["v1,v2"])
    return values


from collections import namedtuple
def test():
    _req_class = namedtuple("Request", ("GET"))
    
    request_1 = _req_class(dict(values="v1, v2,v3"))
    request_2 = _req_class(dict(values="v1,v2,v3 "))
    request_3 = _req_class(dict(values="v1, v2,v3, "))
    
    assert(get(request_1) == ['v1', 'v2', 'v3'])
    assert(get(request_2) == ['v1', 'v2', 'v3'])
    assert(get(request_3) == ['v1', 'v2', 'v3'])
test()

In [24]:
@get_object(MeterInstance)
def post(self, request, *args, **kwargs):
        @optionable
        def get_data(data):
            return data.get("data", None)
        return get_data(request.DATA)\
            .map(
                lambda item:
                handle_error_configuration(item, kwargs["meter"]))\
            .map(
                lambda item:
                Response(dict(detail=item), status=200))\
            .get_or(Response("BadRequest", status=400))

NameError: name 'get_object' is not defined

# Useful libraries

- **Fn.py** (https://github.com/kachayev/fn.py) (Scala functional features ported to Python)
- **Underscore.py** (ported from underscore_js)

# Summary
- Functional programming for writing more succint code
- Change viewpoint: think in terms of function
- Python seems for dummies until you show some cool features
- Python can be slow but with a fine tuning can be more efficien

# Questions?